In [ ]:
from flask import Flask, request, jsonify
import requests
import base64

app = Flask(__name__)

client_Gemini = OpenAI(
    api_key='your_key',
    base_url='your_url'
)

def image_url_to_base64(image_url):
    """
    Downloads an image from the URL and returns its base64 encoding
    """
    response = requests.get(image_url)
    if response.status_code != 200:
        raise ValueError("Failed to download the image")
    return base64.b64encode(response.content).decode('utf-8')

@app.route('/add-object', methods=['POST'])
def add_object():
    try:
        data = request.get_json()
        image_url = data.get('imageUrl')
        addition_text = data.get('additionText')

        if not image_url or not addition_text:
            return jsonify({'error': 'imageUrl or additionText is missing from the request'}), 400

        base64_image = image_url_to_base64(image_url)

        response_Gemini = client_Gemini.chat.completions.create(
            model="gemini-2.0-flash",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": f"Please add the following object to the image: {addition_text}.",
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            },
                        },
                    ],
                }
            ],
        )

        # Assuming the response returns a new image URL or base64 encoded image
        new_image_base64 = response_Gemini.choices[0].message.content

        return jsonify({
            "newImage": new_image_base64
        })

    except Exception as e:
        import traceback
        traceback.print_exc()
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
